In [3]:
from matplotlib import pyplot as plt
from collections import deque
from threading import Lock, Thread

import myo
import numpy as np


class EmgCollector(myo.DeviceListener):
  """
  Collects EMG data in a queue with *n* maximum number of elements.
  """

  def __init__(self, n):
    self.n = n
    self.lock = Lock()
    self.emg_data_queue = deque(maxlen=n)

  def get_emg_data(self):
    with self.lock:
      return list(self.emg_data_queue)

  # myo.DeviceListener

  def on_connected(self, event):
    event.device.stream_emg(True)

  def on_emg(self, event):
    with self.lock:
      self.emg_data_queue.append((event.timestamp, event.emg))


class Plot(object):

  def __init__(self, listener):
    self.n = listener.n
    self.listener = listener
    
    self.df = pd.DataFrame({
    'timestamp': [],
    'emg1': [],
    'emg2': [],
    'emg3': [],
    'emg4': [],
    'emg5': [],
    'emg6': [],
    'emg7': [],
    'emg8': []
    })
    
    self.categories=list(self.df)[:-1]
    self.N = len(self.categories)
    self.values=self.df.loc[0].drop('timestamp').self.values.flatten().tolist()
    self.values += self.values[:1]
    self.angles = [n / float(self.N) * 2 * pi for n in range(self.N)]
    self.angles += self.angles[:1]
    
    self.axes = plt.subplot(111, polar=True)
    plt.xticks(self.angles[:-1], self.categories, color='grey', size=8)
    axes.set_rlabel_position(0)
    plt.yticks([], [], color="grey", size=7)
    plt.ylim(-100,100) # will need an absolute value somewhere because negative values don't work on spider plot
    self.graphs = [axes.plot(np.arange(self.n), np.zeros(self.n))[0]]
    axes.fill(self.angles, self.values, 'b', alpha=0.1)
    plt.ion()

  def update_plot(self):
    emg_data = self.listener.get_emg_data()
    emg_data = np.array([x[1] for x in emg_data]).T
    for g, data in zip(self.graphs, emg_data):
      if len(data) < self.n:
        # Fill the left side with zeroes.
        data = np.concatenate([np.zeros(self.n - len(data)), data])
      self.graphs[0].set_ydata(data)
    plt.draw()

  def main(self):
    while True:
      self.update_plot()
      plt.pause(1.0 / 30)


def main():
  myo.init()
  hub = myo.Hub()
  listener = EmgCollector(512)
  with hub.run_in_background(listener.on_event):
    Plot(listener).main()


if __name__ == '__main__':
  main()

ImportError: No module named myo